In [1]:
### Train an RL agent using DQN architecture in a Unity environment (bananaModel)

In [2]:
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
from collections import deque, defaultdict
import time
import sys
from tqdm import tqdm
from dqnetwork import DQNetwork
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
env = UnityEnvironment(file_name="Banana.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### Action space:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

### State space:
- `37` - dimensions.
- some samples include the agent's velocity.
- ray-based perception in the forward direction of the agent.

### Reward:

- `+1` - Yellow Banana collected.
- `-1` - Blue Banana collected.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


In [6]:
# env_info = env.reset(train_mode=False)[brain_name] # reset the environment
# state = env_info.vector_observations[0]            # get the current state
# score = 0                                          # initialize the score
# i = 0
# while True:
#     i+=1
#     action = np.random.randint(action_size)        # select an action
#     env_info = env.step(action)[brain_name]        # send the action to the environment
#     next_state = env_info.vector_observations[0]   # get the next state
#     reward = env_info.rewards[0]                   # get the reward
#     done = env_info.local_done[0]                  # see if episode has finished
#     score += reward                                # update the score
#     state = next_state                             # roll over the state to next time step
#     if(reward != 0):
#         print(reward)
#     if done:                                       # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))
# print("iterations:",i)

In [7]:
# (37x128) -> (128x64) -> (64x32) -> (32x4)
input_features = [state_size, 128, 64, 32]
output_features = [128, 64, 32, action_size]

In [8]:
model = DQNetwork(input_features, output_features)

In [9]:
### define the agent class

In [10]:
### DEFINE CONSTANTS

In [11]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

In [12]:
import torch
from collections import deque, defaultdict
import torch.optim as optim
from dqnetwork import DQNetwork
import random
from buffer import ReplayBuffer

In [173]:
class Agent():
    """Defines the agent class for DQN using Double Q-learning and Prioritized Experience Replay architecture"""
    def __init__(self, state_size=37, action_size=4, gamma=0.99, lr=0.001, update_every=5):
        """
        Initializes the model.
        ----
        @param:
        1. state_size: size of input # of states.
        2. action_size: size of # of actions.
        3. gamma: discounted return rate.
        4. lr: learning rate for the model.
        5. update_every: update target_model every X time-steps.
        """
        self.state_size = state_size
        self.action_size = action_size
        self.input_features = [state_size, 128, 64, 32]
        self.output_features = [128, 64, 32, action_size]
        self.gamma = gamma
        #Q-network : defines the 2 DQN (using doubling Q-learning architecture via fixed Q target)
        self.qnetwork_local = DQNetwork(self.input_features, self.output_features)
        self.qnetwork_target = DQNetwork(self.input_features, self.output_features)
        #define the optimizer
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=lr)
        
        #replay memory
        self.memory = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE)
        self.update_every = update_every
        self.target_update_counter = 0
        
    def step(self, transition):
        """Performs forward pass of the tranisition
        @param:
        1. transition : (tuple) state, action, reward, next_state, done
        """
        # Save experience in replay memory
        self.memory.add(transition)
        self.target_update_counter = (self.target_update_counter + 1) % self.update_every
        
        #Update target network to local network
        if(self.target_update_counter == 0):
            #primary condition to check if len(buffer) > batch_size
            if(len(self.memory) > BATCH_SIZE):
                experiences = self.memory.sample()
                self.train(experiences)
    
    def get_action(self, state, eps=0.0):
        """
        Determines the action to perform based on epsilon-greedy method
        @param:
        1. state - list of current observations to determine an action for
        2. eps - value for epsilon, stochastic measure.
        @return:
        - action = action chosen by either equiprobably π or using Q-table
        """
        print(state.shape)
        state = torch.from_numpy(state).float().unsqueeze(0)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_val = self.qnetwork_local(state)
        print(action_val)
        self.qnetwork_local.train()#train local network
        
        #Epsilon-greedy selection
        if(random.random() > eps):#exploit
            return np.argmax(action_val.cpu().data.numpy())
        
        return random.choice(np.arange(self.action_size))#explore
    
    def train(self, experiences):
        """
        Train the model.
        @param:
        1. experiences: (Tuple[torch.Variable]) (s,a,r,s',done)
        """
        states, actions, rewards, next_states, done = experiences
        
        #Implement SGD using Adam as regularizer
        Q_targets_next = self.qnetwork_target(next_states.float().unsqueeze(0)).detach().max(1)[0].unsqueeze(0)
        Q_targets = rewards + (self.gamma * Q_targets_next.T * (~done))
        print(states.shape, actions.T.shape)
        Q_expected = self.qnetwork_local(states.float().unsqueeze(-2)).T.detach()
        print(Q_targets.shape, Q_targets_next.shape, Q_expected.shape)
        #set loss as mse.
        loss = F.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        #Update target network using soft update
        self.soft_update(self.qnetwork_local, self.qnetwork_target)
    
    def soft_update(self, local_model, target_model):
        """
        Update target network to local network using a soft update param, τ.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        ------
        @param:
        1. local_model: (DQNetwork) local network model (weights will be copied from)
        2. target_model: (DQNetwork) target network model (weights will be copied into)
        """
        
        target_model = TAU*local_model + (1 - TAU) * target_model
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [174]:
#define the replay buffer class

In [175]:
agent = Agent()

### Implement DQN Agent

In [176]:
def get_epsilon_i(num_episode, epsilon_min=0.01, evaluate=True):
    """Gets the epsilon value given the current episode number"""
    if(evaluate):
        epsilon = max(1/num_episode, epsilon_min)
    return epsilon_min #change this by using a hypertuned value for epsilon when testing

In [177]:
scores = []  # List with all scores per episode
scores_100_mean = 0 # Mean score over the last 100 episodes

NUM_EPISODES = 20
CRIT_SOLVED = 13 #How many Bananas must be collected to succeed?

In [178]:
for epoch in range(1, NUM_EPISODES + 1):

    env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
    state = env_info.vector_observations[0]            # get the current state
    score = 0     # initialize the score
    i = 0
    while True:
        i += 1
        print(i)
        epsilon = get_epsilon_i(epoch)
        action = agent.get_action(state, epsilon)        # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        transition = (state, action, reward, next_state, done) #set transition
        agent.step(transition)                         # step into the next state
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break
    scores.append(score)

1
(37,)
tensor([[ 0.0186, -0.0932,  0.0842,  0.1311]])
2
(37,)
tensor([[ 0.0295, -0.0792,  0.0757,  0.1487]])
3
(37,)
tensor([[ 0.0277, -0.1013,  0.0529,  0.1842]])
4
(37,)
tensor([[ 0.0049, -0.1065,  0.0891,  0.1052]])
5
(37,)
tensor([[ 0.0168, -0.0630,  0.0722,  0.1730]])
6
(37,)
tensor([[-0.0018, -0.0732,  0.0518,  0.1693]])
7
(37,)
tensor([[ 0.0393, -0.0982,  0.0783,  0.1449]])
8
(37,)
tensor([[-0.0218, -0.0380,  0.0845,  0.1548]])
9
(37,)
tensor([[-0.0041, -0.0621,  0.0894,  0.1437]])
10
(37,)
tensor([[ 0.0090, -0.0098,  0.0725,  0.1613]])
11
(37,)
tensor([[ 0.0380, -0.0634,  0.0665,  0.1595]])
12
(37,)
tensor([[ 0.0349, -0.0684,  0.0846,  0.1663]])
13
(37,)
tensor([[ 0.0297, -0.1097,  0.0769,  0.1058]])
14
(37,)
tensor([[ 0.0405, -0.0633,  0.0626,  0.1605]])
15
(37,)
tensor([[-0.0020, -0.1205,  0.0585,  0.2168]])
16
(37,)
tensor([[-0.0062, -0.0857,  0.0732,  0.1328]])
17
(37,)
tensor([[ 0.0350, -0.0764,  0.0703,  0.1502]])
18
(37,)
tensor([[ 0.0298, -0.0660,  0.1072,  0.1435]])
1

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
buffer = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE)

In [32]:
action = [-0.0978,  0.2430, -0.0965,  0.0018]

In [33]:
transition = (state, action, 0.0, state, False)

In [34]:
buffer.add(transition)

In [35]:
for i in range(100):
    buffer.add(transition)

In [36]:
a,b,c,d,e = buffer.sample()

In [37]:
qn = DQNetwork(input_features, output_features)

In [85]:
x = qn(a.float()).detach()

In [109]:
temp = torch.tensor(np.random.rand(3,5))

In [136]:
x.shape, x.unsqueeze(-2).T.shape

(torch.Size([64, 4]), torch.Size([4, 1, 64]))

In [155]:
Q_targets_next = qn(a.float().unsqueeze(0)).detach().max(1)[0].unsqueeze(1)

In [160]:
list(qn.parameters())

[Parameter containing:
 tensor([[-0.0539, -0.0262, -0.0098,  ...,  0.1301,  0.1375,  0.0429],
         [ 0.0349, -0.0146, -0.0793,  ...,  0.0408, -0.0905,  0.1391],
         [ 0.1244, -0.1340,  0.0557,  ...,  0.1120,  0.1314, -0.1232],
         ...,
         [ 0.0220, -0.0387, -0.1189,  ..., -0.0032, -0.1463, -0.0024],
         [ 0.0111, -0.0915, -0.0194,  ..., -0.0254, -0.0364, -0.0644],
         [ 0.1281,  0.0151, -0.0458,  ...,  0.0606, -0.0779, -0.1158]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0425,  0.1634,  0.0830,  0.1200, -0.0655,  0.1611,  0.0110,  0.1185,
         -0.0874, -0.1617, -0.0543,  0.1095,  0.0146,  0.0081, -0.1641, -0.0321,
         -0.0520, -0.0183, -0.1345,  0.1421, -0.0356,  0.0940,  0.1099, -0.0594,
         -0.0359,  0.1353, -0.0715,  0.1567,  0.1133,  0.0945,  0.0994,  0.0249,
         -0.0744, -0.0306, -0.0724,  0.0480, -0.0824, -0.0948,  0.0127, -0.0121,
          0.0019,  0.0223, -0.1622, -0.0886,  0.0951,  0.0889, -0.1583,  0.0644